## More specific using selenium, javascript

In [63]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import random

# Load the website using a webdriver
chrome_options=webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
driver = webdriver.Chrome(chrome_options=chrome_options)

driver.get("https://www.google.com/search?q=hornbach+rottweil&rlz=1C1PRFI_enDE790DE790&oq=hornbach+rottweil&aqs=chrome..69i57.4365j0j9&sourceid=chrome&ie=UTF-8#lrd=0x4790a384d2c9d657:0x67ed9ae12ca5867a,1,,,,")
# Wait for 4 seconds
time.sleep(4)

# Click on google consent
driver.find_element("xpath",'//*[@id="W0wltc"]').click()
# Wait for 5 seconds
time.sleep(random.randint(3,5))

# Click on sort by newest
driver.find_element("xpath",'/html/body/span[2]/g-lightbox/div/div[2]/div[3]/span/div/div/div/div[2]/div[3]/g-scrolling-carousel/div[1]/div/div[2]').click()
# Wait for 3 seconds
time.sleep(3)

# Find the total number of reviews and scroll down
total_number_of_reviews = driver.find_element("xpath",'//*[@id="gsr"]/span[2]/g-lightbox/div/div[2]/div[3]/span/div/div/div/div[1]/div[3]/div[1]/div/span/span').text.split(" ")[0]
total_number_of_reviews = int(total_number_of_reviews.replace('.','')) if '.' in total_number_of_reviews else int(total_number_of_reviews)

# Find scroll-element
scrollable_div = driver.find_element("xpath",'//*[@id="gsr"]/span[2]/g-lightbox/div/div[2]/div[3]/span/div/div/div/div[2]')
# NEED TO CHANGE 100 BACK TO 10 FOR ALL REVIEWS
for i in range(0,(round(total_number_of_reviews/18 - 1))):
        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', 
                scrollable_div)
        time.sleep(random.randint(2,5))

C:\Users\JPSittinger\AppData\Local\Temp\ipykernel_12816\4229483255.py:12: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options)


In [64]:
for elements in driver.find_elements(By.CLASS_NAME, "review-more-link"):
    try:
        elements.click()
    except:
        pass

In [65]:
# wait for elements to load
time.sleep(10)

# extract the HTML using BeautifulSoup
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# do web scraping here

# # close the browser
# browser.close()

In [66]:
review_elements = soup.find_all('div', {'jscontroller': 'fIQYlf'})

In [67]:
# review_elements

In [68]:
# create empty lists to store the scraped data
dates = []
stars = []
reviews = []

In [69]:
# iterate through the review elements and extract the data
for review in review_elements:
    date = review.find('span', {'class': 'dehysf lTi8oc'}).text
    star = review.find('span', {'class': 'Fam1ne EBe2gf'})['aria-label']
    
    dates.append(date)
    stars.append(star)

In [70]:
# Reviews turn out to be tricky because they are hidden in different classes, so separate loop for them:

reviews = []
for review in review_elements:
    review_text = review.find('span', {'class': 'review-full-text'})
    if review_text:
        reviews.append(review_text.text)
    else:
        review_text = review.find('div', {'class': 'Jtu6Td'})
        if review_text:
            reviews.append(review_text.text)
        else:
            reviews.append('None')

In [71]:
# reviews

In [72]:
# Cleaning up the dates list

cleaned_dates = [date.replace("vor ", "").replace("\xa0", " ").replace("einem", "1").replace("einer", "1") for date in dates]
# print(cleaned_dates)

In [73]:
# Cleaning up the stars

cleaned_stars = [star.replace("Bewertung: ", "").replace(",0 von 5,", "") for star in stars]
# print(cleaned_stars)

### Creating relative time columns for the "... time ago" values in cleaned_dates

In [74]:
import pandas as pd
dates_df = {'date': cleaned_dates}
dates_df = pd.DataFrame(dates_df)
# dates_df

In [75]:
# Defining function that calculates the timedelta for certain strings in the date column

from datetime import datetime, timedelta

def get_relative_datetime(time_string):
    current_time = datetime.now()
    time_dict = {"Stunde": timedelta(hours=1),
                 "Stunden": timedelta(hours=1),
                 "Tag": timedelta(days=1),
                 "Tagen": timedelta(days=1),
                 "Woche": timedelta(weeks=1),
                 "Wochen": timedelta(weeks=1),
                 "Monat": timedelta(days=30),
                 "Monaten": timedelta(days=30),
                 "Jahr": timedelta(days=365),
                 "Jahren": timedelta(days=365)}
    num, unit = time_string.split(" ")
    relative_time = current_time - int(num) * time_dict[unit]
    return relative_time

# Testing with hours, days, weeks, months and years (all available time-values)
print(get_relative_datetime("4 Stunden"))
print(get_relative_datetime("2 Tagen"))
print(get_relative_datetime("3 Wochen"))
print(get_relative_datetime("7 Monaten"))
print(get_relative_datetime("2 Jahren"))


2023-01-31 17:52:58.182233
2023-01-29 21:52:58.182233
2023-01-10 21:52:58.182233
2022-07-05 21:52:58.182233
2021-01-31 21:52:58.182233


In [76]:
# Applying the function to all values in the date column and returning (overly specific) relative dates
dates_df["relative_dates"] = dates_df["date"].apply(get_relative_datetime)

In [77]:
# dates_df

In [78]:
# Creating a new column with only year and month of when the review was roughly written
dates_df["year_month"] = dates_df["relative_dates"].apply(lambda x: x.strftime("%m/%Y"))
# dates_df

In [79]:
# Creating a full dataframe of dates, stars and reviews

df_full = dates_df.assign(stars=cleaned_stars, review=reviews)
df_full

,date,relative_dates,year_month,stars,review
0,10 Stunden,2023-01-31 11:52:58.218241,01/2023,4,Bisher gute Erfahrungen gemacht. Hier wird ma...
1,13 Stunden,2023-01-31 08:52:58.218241,01/2023,5,
2,1 Tag,2023-01-30 21:52:58.218241,01/2023,5,
3,1 Tag,2023-01-30 21:52:58.218241,01/2023,5,"Es gibt alles was man braucht, die Preise sind..."
4,1 Tag,2023-01-30 21:52:58.218241,01/2023,3,
...,...,...,...,...,...
1935,3 Jahren,2020-02-01 21:52:58.236257,02/2020,4,
1936,3 Jahren,2020-02-01 21:52:58.236257,02/2020,5,
1937,3 Jahren,2020-02-01 21:52:58.236257,02/2020,5,Einfach nur gutm
1938,3 Jahren,2020-02-01 21:52:58.236257,02/2020,3,


In [80]:
# df_full.head(-60)

In [81]:
# # create a dataframe to store the scraped data
# data = {'date': dates, 'stars': stars, 'review': reviews}
# df = pd.DataFrame(data)

# export the dataframe to a csv file
df_full.to_csv('HORNBACH_Rottweil.csv', index=False)